# preprocessing and combining of supplementary datasets

In [1]:
import numpy as np
import pandas as pd
import os
from os import path
import re

### postal code lookup

In [ ]:
#data source: https://geoportal.statistics.gov.uk/datasets/068ee476727d47a3a7a0d976d4343c59/about
df_postal = pd.read_csv("PCD_OA21_LSOA21_MSOA21_LAD_NOV24_UK_LU.csv", encoding='latin-1')
df_postal = df_postal.rename(columns={"ladcd": "Area code"})
print(df_postal.info())

df_postal_merge = df_postal[["pcd7","pcd8","pcds", "Area code", "ladnm"]]
print()
print(df_postal_merge.head(5))

C:\Users\marti\AppData\Local\Temp\ipykernel_22788\14513260.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_postal = pd.read_csv("PCD_OA21_LSOA21_MSOA21_LAD_NOV24_UK_LU.csv", encoding='latin-1')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709658 entries, 0 to 2709657
Data columns (total 14 columns):
 #   Column     Dtype  
---  ------     -----  
 0   pcd7       object 
 1   pcd8       object 
 2   pcds       object 
 3   dointr     int64  
 4   doterm     float64
 5   usertype   int64  
 6   oa21cd     object 
 7   lsoa21cd   object 
 8   msoa21cd   object 
 9   Area code  object 
 10  lsoa21nm   object 
 11  msoa21nm   object 
 12  ladnm      object 
 13  ladnmw     object 
dtypes: float64(1), int64(2), object(11)
memory usage: 289.4+ MB
None

      pcd7      pcd8     pcds  Area code          ladnm
0  AB1 0AA  AB1  0AA  AB1 0AA  S12000033  Aberdeen City
1  AB1 0AB  AB1  0AB  AB1 0AB  S12000033  Aberdeen City
2  AB1 0AD  AB1  0AD  AB1 0AD  S12000033  Aberdeen City
3  AB1 0AE  AB1  0AE  AB1 0AE  S12000034  Aberdeenshire
4  AB1 0AF  AB1  0AF  AB1 0AF  S12000033  Aberdeen City


In [3]:
#print(df_postal["ladnm"].nunique())

### age data

In [ ]:
#data source: https://www.ons.gov.uk/peoplepopulationandcommunity/personalandhouseholdfinances/incomeandwealth/datasets/householddisposableincomeandinequality

df_age = pd.read_csv("age_gender WIP.csv")
df_age = df_age.rename(columns={"Code": "Area code"})
# print(df_age.info())

df_age_clean = df_age.copy()
for column in df_age.columns[3:]:
    # print(column)
    df_age_clean[column] = df_age_clean[column].apply(lambda x:x.replace(",", "") if "," in str(x) else x)
    df_age_clean[column] = df_age_clean[column].astype('float')

#A small part following code is partially adapted from Saturn Cloud: https://saturncloud.io/blog/how-to-sum-values-of-a-row-of-a-pandas-dataframe-efficiently/
print(df_age_clean.shape)
avg_ages = []
for i in range(df_age_clean.shape[0]):
    i_row = df_age_clean.iloc[i, 4:]
    sums = []
    for j, k in enumerate(i_row):
        sums.append(j * k)

    average_age_i = sum(sums)/df_age_clean.iloc[i, 3]
    avg_ages.append(average_age_i)

print(avg_ages)
print(len(avg_ages))
print(len(avg_ages) == df_age_clean.shape[0])

df_age_clean["Average_age"] = avg_ages

df_age_merge = df_age_clean[["Area code", "Average_age"]]
print()
print(df_age_merge.head(5))

(357, 95)
[40.77242827825027, 40.685707965268286, 41.9177787694827, 42.659971588667034, 42.372442611385466, 41.500115345091544, 38.374883720930235, 45.64521869410344, 43.814815351824734, 41.11224958624608, 40.8925583500692, 41.88237361842733, 37.265180654922176, 42.557772856272635, 42.51300998191197, 42.10233830739563, 40.61870342675898, 37.32100975854428, 42.30758611990918, 43.50914517356919, 42.822366672966986, 44.326882792296914, 41.09275621783049, 41.996143868091096, 46.02820931954932, 38.354673504637574, 38.96085428082928, 40.45937608708998, 33.543025984752994, 37.943290131231464, 38.596262992043734, 36.199161580536845, 41.70099317297902, 40.17149480538297, 40.19207512211555, 41.269737656777934, 42.05543308648818, 39.79568621351169, 42.66767209598498, 47.51903489731174, 39.903798910528, 41.25672064248861, 39.69772977874875, 37.54859952305145, 45.23453132513301, 41.569910960148725, 43.16362547555305, 42.91992742516756, 46.47966562387815, 40.97403818031776, 39.747335832658266, 37.85

### education data

In [ ]:
#data source: https://www.ons.gov.uk/peoplepopulationandcommunity/educationandchildcare/bulletins/educationenglandandwales/census2021

df_edu = pd.read_csv("education IP.csv")
print(df_edu.info())

df_edu_merge = df_edu[["Area code", "Qualification index score"]]
print()
print(df_edu_merge.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 19 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Area code                                              332 non-null    object 
 1   Area name                                              332 non-null    object 
 2   Qualification index score                              332 non-null    float64
 3   Qualification index rank 
(lowest = 1, highest = 331)  332 non-null    object 
 4   Qualification index rank percentile                    332 non-null    object 
 5   No qualifications (number)                             332 non-null    object 
 6   Level 1 and entry level qualifications (number)        332 non-null    object 
 7   Level 2 qualifications (number)                        332 non-null    object 
 8   Apprenticeship (number)                           

### ethnicity data

In [ ]:
# clean data on ethnicity distribution and turn numerical values into float type

#data source: https://www.ons.gov.uk/peoplepopulationandcommunity/culturalidentity/ethnicity/bulletins/ethnicgroupenglandandwales/census2021 

df_ethnicity = pd.read_csv("ethnicities IP.csv")
df_eth_clean = df_ethnicity.copy()
for column in df_ethnicity.columns[2:]:
    # print(column)
    df_eth_clean[column] = df_eth_clean[column].apply(lambda x:x.replace(",", "") if "," in str(x) else x)
    df_eth_clean[column] = df_eth_clean[column].apply(lambda x:x.strip().replace("-", "0") if "-" in str(x) else x)
    df_eth_clean[column] = df_eth_clean[column].astype('float')

#obtaining boolean array for column selection:
results_tmp = []
for p in df_eth_clean.columns:
    if "(number)" in p:
        results_tmp.append(False)
    else:
        results_tmp.append(True)
results_tmp[1] = False
print(results_tmp)

df_eth_merge = df_eth_clean.iloc[:, results_tmp]
print()
print(df_eth_merge.head(5))

[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]

   Area code  Asian, Asian British or Asian Welsh: Bangladeshi\n(percent)  \
0  E06000001                                                0.3             
1  E06000002                                                0.4             
2  E06000003                                                0.1             
3  E06000004                                                0.1             
4  E06000005                                                0.7             

   Asian, Asian British or Asian Welsh: Chinese\n(percent)  \
0                                                0.2         
1                                                0.5         
2                                                0.2         
3                          

### income data

In [ ]:
#data source: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales

df_income = pd.read_csv("income WIP.csv")
df_income = df_income.rename(columns={"Local authority code": "Area code"})
print(df_income.info())
print()

df_inc_clean = df_income.copy()
df_inc_clean["Net annual income after housing costs (£)"] = df_inc_clean["Net annual income after housing costs (£)"].apply(lambda x:x.replace(",", "") if "," in str(x) else x)
df_inc_clean["Net annual income after housing costs (£)"] = df_inc_clean["Net annual income after housing costs (£)"].astype('float')

df_inc_clean = df_inc_clean[["Area code", "Net annual income after housing costs (£)"]]

print(df_inc_clean.head())

local_agg = df_inc_clean.groupby(["Area code"]).mean()
print("means: ")
print(local_agg)
print(type(local_agg))

print("final results: ")
df_inc_merge = local_agg
print(df_inc_merge.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7201 entries, 0 to 7200
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   MSOA code                                  7201 non-null   object
 1   MSOA name                                  7201 non-null   object
 2   Area code                                  7201 non-null   object
 3   Local authority name                       7201 non-null   object
 4   Region code                                7201 non-null   object
 5   Region name                                7201 non-null   object
 6   Net annual income after housing costs (£)  7201 non-null   object
 7   Upper confidence limit (£)                 7201 non-null   object
 8   Lower confidence limit (£)                 7201 non-null   object
 9   Confidence interval (£)                    7201 non-null   object
dtypes: object(10)
memory usage: 562.7+ K

### merging data

In [9]:
merged = df_postal_merge.merge(df_age_merge, how='inner', on='Area code')
print(merged.info())
print(merged.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396067 entries, 0 to 2396066
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pcd7         object 
 1   pcd8         object 
 2   pcds         object 
 3   Area code    object 
 4   ladnm        object 
 5   Average_age  float64
dtypes: float64(1), object(5)
memory usage: 109.7+ MB
None
      pcd7      pcd8     pcds  Area code      ladnm  Average_age
0  AL1 1AA  AL1  1AA  AL1 1AA  E07000240  St Albans    40.449336
1  AL1 1AB  AL1  1AB  AL1 1AB  E07000240  St Albans    40.449336
2  AL1 1AD  AL1  1AD  AL1 1AD  E07000240  St Albans    40.449336
3  AL1 1AE  AL1  1AE  AL1 1AE  E07000240  St Albans    40.449336
4  AL1 1AF  AL1  1AF  AL1 1AF  E07000240  St Albans    40.449336


In [10]:
merged = merged.merge(df_edu_merge, how='inner', on='Area code')
print(merged.info())
print(merged.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312399 entries, 0 to 2312398
Data columns (total 7 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   pcd7                       object 
 1   pcd8                       object 
 2   pcds                       object 
 3   Area code                  object 
 4   ladnm                      object 
 5   Average_age                float64
 6   Qualification index score  float64
dtypes: float64(2), object(5)
memory usage: 123.5+ MB
None
      pcd7      pcd8     pcds  Area code      ladnm  Average_age  \
0  AL1 1AA  AL1  1AA  AL1 1AA  E07000240  St Albans    40.449336   
1  AL1 1AB  AL1  1AB  AL1 1AB  E07000240  St Albans    40.449336   
2  AL1 1AD  AL1  1AD  AL1 1AD  E07000240  St Albans    40.449336   
3  AL1 1AE  AL1  1AE  AL1 1AE  E07000240  St Albans    40.449336   
4  AL1 1AF  AL1  1AF  AL1 1AF  E07000240  St Albans    40.449336   

   Qualification index score  
0                       2.98 

In [11]:
merged = merged.merge(df_inc_merge, how='inner', on='Area code')
print(merged.info())
print(merged.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165695 entries, 0 to 2165694
Data columns (total 8 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   pcd7                                       object 
 1   pcd8                                       object 
 2   pcds                                       object 
 3   Area code                                  object 
 4   ladnm                                      object 
 5   Average_age                                float64
 6   Qualification index score                  float64
 7   Net annual income after housing costs (£)  float64
dtypes: float64(3), object(5)
memory usage: 132.2+ MB
None
      pcd7      pcd8     pcds  Area code      ladnm  Average_age  \
0  AL2 1BB  AL2  1BB  AL2 1BB  E07000098  Hertsmere    40.680154   
1  AL2 1BT  AL2  1BT  AL2 1BT  E07000098  Hertsmere    40.680154   
2  AL2 1BU  AL2  1BU  AL2 1BU  E07000098  Hertsmere    40.680154 

In [12]:
merged_complete = merged.merge(df_eth_merge, how='inner', on='Area code')
print(merged_complete.info())
print(merged_complete.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165695 entries, 0 to 2165694
Data columns (total 27 columns):
 #   Column                                                                            Dtype  
---  ------                                                                            -----  
 0   pcd7                                                                              object 
 1   pcd8                                                                              object 
 2   pcds                                                                              object 
 3   Area code                                                                         object 
 4   ladnm                                                                             object 
 5   Average_age                                                                       float64
 6   Qualification index score                                                         float64
 7   Net annual income after hou

### export

In [13]:
merged_complete.to_csv("supplementary_materials.csv")